In [1]:
from fov import FOV
import useq
from useq import MDAEvent
from queue import Queue
import numpy as np
from matplotlib import pyplot as plt
from napari import Viewer
import numpy as np
from queue import Queue
from pymmcore_plus import CMMCorePlus
from useq import MDAEvent
import useq
from useq._channel import Channel
from MDAEngine_DMD import MDAEngine_DMD
from controller import Controller, Analyzer
import pandas as pd
import random
from utils import ImgType, MetadataDict
from stimulation import StimExtraParameters
from dmd import DMD
from hardware import load_config
import os
import tempfile
from datetime import datetime
import pymmcore
import napari
from tracking import TrackerTrackpy
from utils import create_folders
import numpy as np

## Load uManager and set default devices

In [2]:
from pymmcore_plus import CMMCorePlus
import numpy as np
import pymmcore_plus
import numpy as np
import time 

mmc = pymmcore_plus.CMMCorePlus()

#mmc.setDeviceAdapterSearchPaths(['C:\\Users\\micromanager\\AppData\\Local\\pymmcore-plus\\pymmcore-plus\\mm\\Micro-Manager_2.0.3_20230925'])
mmc.loadSystemConfiguration(fileName='C:/Users/micromanager/Documents/lhinder/code/rtm-pymmcore/local/local_config.cfg')


In [3]:
# Create a new log file in the temp directory
path = 'F:/lhinder/tmp/'

create_folders(path,['stim','raw','labels','stim_mask','tracks','labels_rings','particles'])

#TODO: create dir and file

log_file = os.path.join(path, "log.txt")

# Get the current date and time
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print(f"Logfile: {log_file}")

# Open the log file in append mode and write the date and time
with open(log_file, "w") as file: #w: write (delete old content), a: append
    file.write(f"{current_time}\n")
    file.write(f"User ID: {mmc.getUserId()}")

    file.write(f"PyMMCore version {pymmcore.__version__}\n")
    file.write(f"{mmc.getAPIVersionInfo()}")
    file.write(f"{mmc.getVersionInfo()}\n\n")

    file.write("Loaded devices:\n")
    file.write("\n".join(str(device) for device in mmc.getLoadedDevices()))
    file.write("\n\n")

Directory F:/lhinder/tmp/stim already exists
Directory F:/lhinder/tmp/raw already exists
Directory F:/lhinder/tmp/labels already exists
Directory F:/lhinder/tmp/stim_mask already exists
Directory F:/lhinder/tmp/tracks already exists
Directory F:/lhinder/tmp/labels_rings already exists
Directory F:/lhinder/tmp/particles already exists
Logfile: F:/lhinder/tmp/log.txt


## Create list with FOVs

In [4]:
# get current position
pos = (mmc.getXPosition(),mmc.getYPosition())
pos_list = [pos,pos]

In [5]:
print(mmc.getXPosition())
print(mmc.getYPosition())

6.400000095367432
-5.500000081956387


## Create a DF with all planned acquisitions and stimulations

In [6]:
df_acquire = pd.DataFrame(columns=['fov', 'timestep', 'time','time_experiment', 'treatment', 'acquired','stim', 'channels', 'channel_stim'])

time_between_frames = 5 #time in seconds between frames
stim_timesteps = [2,3]  # list of timesteps
#stim_timesteps= []  # list of timesteps

timesteps = range(5)  # 0-20
treatments = ['stim_top', 'stim_mid',]  # list of treatments
channels = [['miRFP','mCherry']]
channels_exposure = [[200,100]]
channel_stim = ['mCitrine']
channel_stim_exposure = [100]

fovs:list[FOV] = []

for i,pos in enumerate(pos_list):
    print(pos)
    fov = FOV(pos=pos,
              index =i,
              path=path,
              metadata={},
              properties={'stim_property': 'top'},
              )
    fovs.append(fov)

# Loop over the FOVs and randomly assign one of the treatments to it
treatments_shuffled = treatments.copy()
random.shuffle(treatments_shuffled)
for fov in fovs:
    treatment = treatments_shuffled[fov.index % len(treatments_shuffled)]
    for timestep in timesteps:
        new_row = { 'fov_object': fov,
                    'fov':fov.index,
                    'timestep': timestep,
                    'time': timestep*time_between_frames,
                    'treatment': treatment,
                    'acquired': False,
                    'stim': timestep in stim_timesteps,
                    'channels': channels,
                    'channels_exposure':channels_exposure,
                    'channel_stim' : channel_stim,
                    'channel_stim_exposure' : channel_stim_exposure,
                    'fname' : f'{str(fov.index).zfill(3)}_{str(timestep).zfill(5)}',
                    
                    ## Custom parameters for the stimulation
                    'offset_x': 0, #offset for the stimulation mask
                    'offset_y': 10, #offset for the stimulation mask
                    'radius': 10, #radius of the stimulation mask
                    }
        df_acquire = pd.concat([df_acquire, pd.DataFrame(new_row, index=[0])])


df_acquire = df_acquire.sort_values(by=['timestep', 'fov'])

df_acquire

(6.400000095367432, -5.500000081956387)
(6.400000095367432, -5.500000081956387)


,fov,timestep,time,time_experiment,treatment,acquired,stim,channels,channel_stim,fov_object,channels_exposure,channel_stim_exposure,fname,offset_x,offset_y,radius
0,0,0,0,NaN,stim_top,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x000001E20761DB80>,"[200, 100]",100.0,000_00000,0.0,10.0,10.0
0,1,0,0,NaN,stim_mid,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x000001E207622610>,"[200, 100]",100.0,001_00000,0.0,10.0,10.0
0,0,1,5,NaN,stim_top,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x000001E20761DB80>,"[200, 100]",100.0,000_00001,0.0,10.0,10.0
0,1,1,5,NaN,stim_mid,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x000001E207622610>,"[200, 100]",100.0,001_00001,0.0,10.0,10.0
0,0,2,10,NaN,stim_top,False,True,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x000001E20761DB80>,"[200, 100]",100.0,000_00002,0.0,10.0,10.0
0,1,2,10,NaN,stim_mid,False,True,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x000001E207622610>,"[200, 100]",100.0,001_00002,0.0,10.0,10.0
0,0,3,15,NaN,stim_top,False,True,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x000001E20761DB80>,"[200, 100]",100.0,000_00003,0.0,10.0,10.0
0,1,3,15,NaN,stim_mid,False,True,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x000001E207622610>,"[200, 100]",100.0,001_00003,0.0,10.0,10.0
0,0,4,20,NaN,stim_top,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x000001E20761DB80>,"[200, 100]",100.0,000_00004,0.0,10.0,10.0
0,1,4,20,NaN,stim_mid,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x000001E207622610>,"[200, 100]",100.0,001_00004,0.0,10.0,10.0


## Run on virtual system 

### TEST MODE

## Run on system with DMD

In [7]:
from add_frame import ImageProcessingPipeline
from segmentation import SegmentatorStardist
from controller import Controller, Analyzer


segmentator = SegmentatorStardist('2D_versatile_fluo')
stimulator = StimExtraParameters()
tracker = TrackerTrackpy(search_range=10,memory=3,adaptive_stop=1,adaptive_step=0.8)
pipeline = ImageProcessingPipeline(segmentator,stimulator,tracker)
analyzer = Analyzer(pipeline)

dmd = DMD(mmc)

dmd.calibrate()

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


array([[ 1.04000000e+02,  2.08000000e+01, -4.95320000e+04],
       [-5.03636364e+01, -1.68000000e+01,  2.82565455e+04]])

In [8]:
mmc.mda.set_engine(MDAEngine_DMD(dmd))

# Apply the custom acquisition engine
# Register the custom engine with the runner
#core.mda.set_engine(MDAEngine_DMD(core))

# create the Queue that will hold the MDAEvents
queue = Queue()


controller = Controller(analyzer, mmc , queue, dmd)
# Start the acquisitiondd
controller.run(df_acquire=df_acquire)

2023-10-19 12:02:49,750 - pymmcore-plus - INFO - (_runner.py:216) MDA Started: GeneratorMDASequence()
2023-10-19 12:02:49,753 - pymmcore-plus - INFO - (_runner.py:184) channel=Channel(config='miRFP') exposure=200.0 min_start_time=0.0 x_pos=6.400000095367432 y_pos=-5.500000081956387 metadata={'fov': 0, 'timestep': 0, 'time': 0, 'time_experiment': nan, 'treatment': 'stim_top', 'acquired': False, 'stim': False, 'channels': ['miRFP', 'mCherry'], 'channel_stim': 'mCitrine', 'fov_object': <fov.FOV object at 0x000001E20761DB80>, 'channels_exposure': [200, 100], 'channel_stim_exposure': 100.0, 'fname': '000_00000', 'offset_x': 0.0, 'offset_y': 10.0, 'radius': 10.0, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': False, 'stim_mask': array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255,

Current timestep: 0
Current timestep: 1


2023-10-19 12:02:50,701 - pymmcore-plus - INFO - (_runner.py:184) channel=Channel(config='mCherry') exposure=100.0 min_start_time=0.0 x_pos=6.400000095367432 y_pos=-5.500000081956387 metadata={'fov': 0, 'timestep': 0, 'time': 0, 'time_experiment': nan, 'treatment': 'stim_top', 'acquired': False, 'stim': False, 'channels': ['miRFP', 'mCherry'], 'channel_stim': 'mCitrine', 'fov_object': <fov.FOV object at 0x000001E20761DB80>, 'channels_exposure': [200, 100], 'channel_stim_exposure': 100.0, 'fname': '000_00000', 'offset_x': 0.0, 'offset_y': 10.0, 'radius': 10.0, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': True, 'stim_mask': array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), 'channel': 'mCherry'}
2023-10-19 12:02:51,619 - pymmcore-plus - INFO - (_r

Current timestep: 2


2023-10-19 12:02:54,754 - pymmcore-plus - INFO - (_runner.py:184) channel=Channel(config='miRFP') exposure=200.0 min_start_time=5.0 x_pos=6.400000095367432 y_pos=-5.500000081956387 metadata={'fov': 0, 'timestep': 1, 'time': 5, 'time_experiment': nan, 'treatment': 'stim_top', 'acquired': False, 'stim': False, 'channels': ['miRFP', 'mCherry'], 'channel_stim': 'mCitrine', 'fov_object': <fov.FOV object at 0x000001E20761DB80>, 'channels_exposure': [200, 100], 'channel_stim_exposure': 100.0, 'fname': '000_00001', 'offset_x': 0.0, 'offset_y': 10.0, 'radius': 10.0, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': False, 'stim_mask': array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), 'channel': 'miRFP'}
2023-10-19 12:02:55,639 - pymmcore-plus - INFO - (_runn

Current timestep: 3


2023-10-19 12:03:06,278 - pymmcore-plus - INFO - (_runner.py:184) channel=Channel(config='miRFP') exposure=200.0 min_start_time=15.0 x_pos=6.400000095367432 y_pos=-5.500000081956387 metadata={'fov': 0, 'timestep': 3, 'time': 15, 'time_experiment': nan, 'treatment': 'stim_top', 'acquired': False, 'stim': True, 'channels': ['miRFP', 'mCherry'], 'channel_stim': 'mCitrine', 'fov_object': <fov.FOV object at 0x000001E20761DB80>, 'channels_exposure': [200, 100], 'channel_stim_exposure': 100.0, 'fname': '000_00003', 'offset_x': 0.0, 'offset_y': 10.0, 'radius': 10.0, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': False, 'stim_mask': array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), 'channel': 'miRFP'}
2023-10-19 12:03:07,497 - pymmcore-plus - INFO - (_run

Current timestep: 4


2023-10-19 12:03:12,402 - pymmcore-plus - INFO - (_runner.py:184) channel=Channel(config='miRFP') exposure=200.0 min_start_time=20.0 x_pos=6.400000095367432 y_pos=-5.500000081956387 metadata={'fov': 0, 'timestep': 4, 'time': 20, 'time_experiment': nan, 'treatment': 'stim_top', 'acquired': False, 'stim': False, 'channels': ['miRFP', 'mCherry'], 'channel_stim': 'mCitrine', 'fov_object': <fov.FOV object at 0x000001E20761DB80>, 'channels_exposure': [200, 100], 'channel_stim_exposure': 100.0, 'fname': '000_00004', 'offset_x': 0.0, 'offset_y': 10.0, 'radius': 10.0, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': False, 'stim_mask': array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), 'channel': 'miRFP'}
2023-10-19 12:03:13,445 - pymmcore-plus - INFO - (_ru

In [ ]:
mmc.unloadAllDevices()

In [ ]:
from add_frame import ImageProcessingPipeline
from segmentation import SegmentatorStardist
from pymmcore_plus.mda import MDAEngine
from stimulation import StimExtraParameters, StimCircle
from controller import Analyzer

segmentator = SegmentatorStardist('2D_versatile_fluo')
stimulator = StimExtraParameters()
stimulator = StimCircle()
tracker = TrackerTrackpy()
pipeline = ImageProcessingPipeline(segmentator,stimulator,tracker)
analyzer = Analyzer(pipeline)


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [ ]:
import napari
import zarr
import glob
from skimage.io import imread
from glob import glob
import dask.array as da
from dask import delayed
import os
import numpy as np
from magicgui import magicgui
import pandas as pd



def tiff_to_lazy_da(path,folder,fov, zfill = 2):
    '''Read in all tiff files form the same FOV in a folder and load them lazily with dask. '''
    file_name_pattern = str(fov).zfill(zfill)+"_*.tiff"
    filenames = sorted(glob(path + os.path.join(str(folder),file_name_pattern)))
    # read the first file to get the shape and dtype
    # ASSUMES THAT ALL FILES SHARE THE SAME SHAPE and TYPE

    sample = imread(filenames[0])
    
    lazy_imread = delayed(imread)  # lazy reader
    lazy_arrays = [lazy_imread(fn) for fn in filenames]
    dask_arrays = [
        da.from_delayed(delayed_reader, shape=sample.shape, dtype=sample.dtype)
        for delayed_reader in lazy_arrays
    ]
    # Stack into one large dask.array
    stack = da.stack(dask_arrays, axis=0)
    stack = np.squeeze(stack)
    return stack

project_path = '/Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_352/'
stack_raw = tiff_to_lazy_da(project_path, "raw", 0,zfill=2)
stack_stim = tiff_to_lazy_da(project_path, "stim", 0,zfill=2)

In [ ]:

from controller_simulation import Controller, Analyzer
from dmd import DMD

dmd = DMD(mmc, test_mode = True)
mda_engine_dmd = MDAEngine_DMD(dmd)

mmc.mda.set_engine(mda_engine_dmd)

queue = Queue()

### RUN THIS IF YOU WANT TO TEST THE ACQUISITION WITHOUT ACTUAL HARDWARE
analyzer = Analyzer(pipeline)
controller = Controller(analyzer, mmc, queue,stack_raw=stack_raw,stack_stim=stack_stim)


# Start the acquisition
controller.run(df_acquire)


NameError: name 'stack_raw' is not defined

In [ ]:
mmc.getLoadedDevices()

('Core',)

In [ ]:
def normalize_mi_ma(x, mi, ma, clip=False, eps=1e-20, dtype=np.float32):
    if dtype is not None:
        x   = x.astype(dtype,copy=False)
        mi  = dtype(mi) if np.isscalar(mi) else mi.astype(dtype,copy=False)
        ma  = dtype(ma) if np.isscalar(ma) else ma.astype(dtype,copy=False)
        eps = dtype(eps)

def normalize(x, pmin=3, pmax=99.8, axis=None, clip=False, eps=1e-20, dtype=np.float32):
    """Percentile-based image normalization."""

    mi = np.percentile(x.flatten(),pmin)
    ma = np.percentile(x.flatten(),pmax)
    return normalize_mi_ma(x, mi, ma, clip=clip, eps=eps, dtype=dtype)

normalize(np.ones((1000,1000)))
print(np.ones((1000,1000)).flatten().shape)

(1000000,)
